# Задание 1

Город разбит на квадраты и для каждого квадрата известно количество людей, живущих и работающих на территории этого квадрата. Эти данные находятся в файле `data.csv`. Например, на территории квадрата `85881` живет 101 человек и работает 28 человек.

Также город разбит на административные районы, их существенно меньше. Для каждого квадрата известно то, с какими районами он пересекается и по какой части своей площади. Эти данные находятся в файле `area2district.csv`. Например, примерно 38% площади квадрата `91422` составляет район `55`.

От Вас требуется предложить способ (и реализовать его) расчета количества людей, живущих и работающих в каждом административном районе соответственно. Можно считать, что внутри каждого квадрата люди распределены равномерно.

Требования:
   * В каждом районе должно жить и работать целое неотрицательное количество людей
   * Сумма проживающих и работающих жителей города не должна измениться

В данной задаче нет строго критерия оценивания, но важно сделать упор на скорость работы, в частности не рекомендуется пользоваться циклами `for`. 

In [1]:
import pandas as pd

df_cell2adm = pd.read_csv('area2district.csv', sep=',')
df_data = pd.read_csv('data.csv', sep=',')

Для начала посчитаем сумму работающих и сидящих дома людей для всех квадратов, чтобы потом сравнить с результатом:

In [168]:
df_cell2adm.head()

,areaid,districtid,percent
0,89012,478,1.000000
1,123048,55,0.984496
2,34536,7,1.000000
3,91422,55,0.380014
4,106142,55,0.912069


In [4]:
df_data.sum()

areaid    13901796637
home         19160892
job           9111365
dtype: int64

Соединим таблицы квадратов и районов, используя areaid как ключ

In [5]:
df_data_cell = df_data.join(df_cell2adm.set_index('areaid'), on='areaid')

In [11]:
df_data_cell.sort_values(by=['areaid'])

,areaid,home,job,districtid,percent
125505,0,290,109,120,1.000000
104867,1,16,8,476,1.000000
46982,2,2,0,90,0.638421
46982,2,2,0,303,0.361579
121903,4,18,9,146,1.000000
...,...,...,...,...,...
142689,190236,37,16,355,1.000000
29198,190237,24,8,168,1.000000
40118,190239,2,0,296,1.000000
143448,190240,13,8,465,1.000000


Посчитаем долю людей для каждого района в одном квадрате, сколько не хватает, допишем

In [106]:
def calculate(num, div):
    '''Функция для промежуточных вычислений разницы округления
    num - разница между эталоном и округленным значением
    div - количество районов'''
    return [num // div + (1 if x < num % div else 0)  for x in range (div)]

In [104]:
# def count_in_square(data):
#     """доля людей для каждого района в одном квадрате"""
#     person_home = round(data['home'] * data ['percent']) 
#     person_job = round(data['job'] * data['percent'])
#     if data['home'] != sum(person_home) :
#         delta_home = sum(person_home) - data['home']
#         person_home + delta_home
#     if data['job'] != sum(person_job):
#         delta_job = sum(person_job) - data['job']
#         person_job +=delta
#     return pd.Series([sum(person_home), sum(person_job)], index=['person_home', 'person_job'])

In [159]:
def count_in_square(data):
    person_home = round(data['home'] * data ['percent']) 
    person_job = round(data['job'] * data['percent'])
    delta_home = sum(person_home) - data['home']
    delta_job = sum(person_job) - data['job']
    district_num_list_home = calculate(delta_home.iloc[0], len(data))
    district_num_list_job = calculate(delta_job.iloc[0], len(data))
    return pd.Series([sum(person_home) + district_num_list_home[0], sum(person_job)+ district_num_list_job[0]], index=['person_home', 'person_job'])
    #TODO добавить полученные списки к серии из количества людей

In [160]:
square_count = df_data_cell.groupby(by='areaid').apply(count_in_square)
square_count

,person_home,person_job
areaid,,
0,290.0,109.0
1,16.0,8.0
2,2.0,0.0
4,18.0,9.0
5,34.0,16.0
...,...,...
190236,37.0,16.0
190237,24.0,8.0
190239,2.0,0.0


Сгруппируем таблица по районам и посмотрим на количество жителей

In [195]:
joined = square_count.join(df_cell2adm.set_index('areaid'), on='areaid').groupby(by='districtid')
joined.head()

,person_home,person_job,districtid,percent
areaid,,,,
0,290.0,109.0,120,1.000000
1,16.0,8.0,476,1.000000
2,2.0,0.0,303,0.361579
2,2.0,0.0,90,0.638421
4,18.0,9.0,146,1.000000
...,...,...,...,...
53214,1122.0,524.0,69,0.297238
53919,3802.0,1008.0,304,1.000000
56345,3013.0,1146.0,246,0.998019


In [161]:
df_data.sort_values(by=['areaid'])

,areaid,home,job
125505,0,290,109
104867,1,16,8
46982,2,2,0
121903,4,18,9
15265,5,34,16
...,...,...,...
142689,190236,37,16
29198,190237,24,8
40118,190239,2,0
143448,190240,13,8


Теперь сравним данные посчитанные на основе функции и с первоначальными данными по квадратам

In [182]:
# square_count['person_home'].isin(df_data['home']).value_counts()

True     146276
False        20
Name: person_home, dtype: int64

In [183]:
# square_count['person_job'].isin(df_data['job']).value_counts()

True     146266
False        30
Name: person_job, dtype: int64

Посмотрим на значения, которые отличаются от первоначальных

In [164]:
df_1notin2 = square_count[~(square_count['person_job'].isin(df_data['job']))].reset_index(drop=False)
df_1notin2

,areaid,person_home,person_job
0,664,2734.0,893.0
1,7311,2075.0,6296.0
2,8361,2799.0,2110.0
3,9100,2709.0,3561.0
4,20666,2688.0,4331.0
5,21000,3619.0,3635.0
6,31616,1809.0,4420.0
7,42601,1610.0,8143.0
8,50928,3785.0,2615.0
9,58649,5346.0,1961.0


In [165]:
df_1notin2 = square_count[~(square_count['person_home'].isin(df_data['home']))].reset_index(drop=False)
df_1notin2

,areaid,person_home,person_job
0,8361,2799.0,2110.0
1,22062,3543.0,5737.0
2,22682,1862.0,2349.0
3,30287,2520.0,1073.0
4,32107,4331.0,1545.0
5,42601,1610.0,8143.0
6,43036,6332.0,2115.0
7,48924,2389.0,1694.0
8,58649,5346.0,1961.0
9,61726,4014.0,3063.0


Напишем функцию, которая посчитает количество работающих и сидящих дома людей в серии с учетом процента вхождения квадрата в район

In [73]:
def data_with_percent(data):
    result_home = data['home'] * data['percent']
    result_job = data['job'] * data['percent']
    return pd.Series([sum(result_home), sum(result_job)], index=['result_home', 'result_job'])

Сгруппируем данные по районам и применим функцию, написанную выше

In [75]:
df_group = df_data_cell.groupby(by='districtid').apply(data_with_percent)

In [76]:
df_group

,result_home,result_job
districtid,,
1,32625.162519,13566.430665
2,3716.605467,535.239616
3,46330.593462,18062.574715
4,3966.771421,1243.848413
5,4833.746494,391.347340
...,...,...
474,5557.526529,1934.038227
476,53653.821433,19837.551282
477,125351.117556,76267.900661


In [77]:
df_without = df_data_cell.groupby(by='districtid')
df_without.sum()

,areaid,home,job,percent
districtid,,,,
1,25113398,39779,16457,192.368235
2,109804374,4296,709,1064.225784
3,4759598,86584,32279,31.352369
4,27308777,4700,1493,246.464441
5,126152491,5226,492,1273.438838
...,...,...,...,...
474,9424741,6316,2182,73.165219
476,28955987,65924,25186,240.762178
477,30143873,167040,96951,267.201569


In [78]:
df_group.round()

,result_home,result_job
districtid,,
1,32625.0,13566.0
2,3717.0,535.0
3,46331.0,18063.0
4,3967.0,1244.0
5,4834.0,391.0
...,...,...
474,5558.0,1934.0
476,53654.0,19838.0
477,125351.0,76268.0


Видим, что если пользоваться таблицей данных после округления, то у нас получается перебор на 13 человек для сидящих дома и на 2 для работающих, как с этим жить, пока непонятно, возможно существуют методы свести эту погрешность к нулю

In [21]:
(19160892+9111365) - (19160905+9111367)

-15

Сделаем джойн двух фреймов, в которых информация оригинальная по квадратам и после взятия процентов и округления

In [91]:
join_square = square_count.join(df_data.set_index('areaid'), on='areaid')

In [90]:
join_square.loc[join_square['person_job'] - join_square['job'] !=0]

,person_home,person_job,home,job
areaid,,,,
664,2734.0,892.0,2734,891
3097,965.0,230.0,965,229
4917,57.0,15.0,56,16
5258,300.0,87.0,300,88
5261,2721.0,801.0,2720,802
...,...,...,...,...
187525,1447.0,408.0,1448,409
187907,34.0,17.0,34,18
187921,2584.0,1406.0,2584,1405


Мы получили результирующию таблицу, с помощью которой сможем избавиться от ошибки округления, необходимо найти разность с эталоном и эту разницу равномерно распределить по районам

In [184]:
df_data.sum()

areaid    13901796637
home         19160892
job           9111365
dtype: int64

person_home    26678581.0
person_job     13318505.0
percent          146296.0
dtype: float64